In [1]:
!pip install -q -U transformers==4.55.2 trl==0.21.0 datasets==4.0.0 huggingface_hub==0.34.4 accelerate==1.10.0 peft==0.17.0 bitsandbytes==0.47.0 wandb==0.21.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 83.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 70.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.1 MB/s eta 0:00

In [2]:
import os
import torch
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig, setup_chat_format
from transformers import BitsAndBytesConfig

2025-08-22 13:32:40.528194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755869560.730363      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755869560.787467      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

hf_token = UserSecretsClient().get_secret("HF_TOKEN")

login(token=hf_token)

In [4]:
MODEL_NAME = "google/gemma-2b-it"
NEW_MODEL_REPO = "Yanmife/gemma-2b-it-health-convo-big"
WANDB_PROJECT = "Gemma-2-it-big"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
# Load in fp16 (or bf16 if supported by GPU)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=False,
    low_cpu_mem_usage=True,
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,            # rank
    lora_alpha=64,
    target_module="all-linear"
    # target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


trainable params: 78,446,592 || all params: 2,584,619,008 || trainable%: 3.0351


In [ ]:
model = torch.compile(model)     

# helps the model to run faster. was referenced in Andrej Karpathy's tutorial. Always use in fintuning or building from scratch

In [10]:
from datasets import load_dataset

# Example dataset (replace with yours)
health_ds = load_dataset("Oluwadara/health_conversations", split="train")
print(health_ds)

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 3000
})


In [11]:
print(health_ds[0])

{'Description': 'Can blood pressure medication be stopped to check improvement in bp levels?', 'Patient': "I'm 35, BP 150/100 without medicine, never smoke, drink, BMIMy hdl:45, LDL:107, Total Cholesterol:168, Triglyceride:95, Blood Sugar(Fasting):83 (all are without medicine) My question is, should I stop this medicine to see the afffect. it is worth to mention that recently I've increased my physical activity,", 'Doctor': 'Hello,Thanks for writing to Health Care Magic, I am Dr Asad Riaz, I have closely read your question and I understand your concerns, I will hereby guide you regarding your health related problem.BP is one major risk factor for major complication like MI or stroke...if pt have high BP as u r having ..1st we need to do life style modification to see whteher it work to de BP or not in whch i advice pt to lower salt intake,drinkng dec weight n exercise for a period of 6 month n still if pt have high BP then need to add med to control it so we prevent major comlication..

In [12]:
health_ds["Description"][0:3]

['Can blood pressure medication be stopped to check improvement in bp levels?',
 'What causes blood in urine?',
 'What causes swelling and blisters around lips of my daughter?']

In [13]:
ds_2 = load_dataset("Mohammed-Altaf/medical-instruction-120k")

README.md:   0%|          | 0.00/965 [00:00<?, ?B/s]

medicare_110k_train.json:   0%|          | 0.00/126M [00:00<?, ?B/s]

medicare_110k_test.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/106556 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5609 [00:00<?, ? examples/s]

In [14]:
ds_2

DatasetDict({
    train: Dataset({
        features: ['Conversation'],
        num_rows: 106556
    })
    test: Dataset({
        features: ['Conversation'],
        num_rows: 5609
    })
})

In [15]:
ds_2["train"]

Dataset({
    features: ['Conversation'],
    num_rows: 106556
})

In [16]:
ds_2["train"][0]

{'Conversation': 'The conversation between human and AI assistant.\n[|Human|] Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to life. now he isnot conscious.he is in ICU.his doctor said his conscious rate is 5.what do you think about his condition?If it is needed I can send you his cardiograf and blood test and the drugs they are using for him in ICU.Kindly tell me can we carry him to a better hospital in Tehran(It takes 4 hours to reach there)?\n[|AI|]  Hello Addie! Welcome and thank you for asking on Chat Doctor! I passed carefully through your question and would explain that your father clinical situation could be related to prolonged brain hypoxia, due to the cardiac arrest. The time (40 minutes) needed for his resuscitation is long and may have caused irreversible damage to his brain, due to low blood flow (also called hypoxic encephalopathy). For this reason, I recommend consulting 

In [17]:
import re
import pandas as pd
from datasets import Dataset

# This is your new "mapper" function.
# It is designed to work on a BATCH of examples.
def process_data_batch(batch):
    # This will hold the processed data for the entire batch
    processed_batch = {
        'input': [],
        'output': [],
    }
    
    # Iterate through the rows in the batch
    # 'batch' is a dictionary of lists, so we use indices
    for i in range(len(batch['Conversation']) if 'Conversation' in batch else len(batch['Description'])):
        
        if 'Conversation' in batch:
            # Handling Dataset 2 format
            conv = batch['Conversation'][i]
            
            # Use regex to parse the conversation
            parts = re.split(r'\[\|Human\|\]|\[\|AI\|\]', conv)
            if len(parts) >= 3:
                human_text = parts[1].strip()
                ai_text = parts[2].strip()
            else:
                human_text = conv
                ai_text = ""
            
            # Clean the text segments
            human_text_cleaned = re.sub(r'([.?!])([A-Z])', r'\1 \2', human_text)
            human_text_cleaned = re.sub(r'\s+', ' ', human_text_cleaned).strip()
            
            ai_text_cleaned = re.sub(r'([.?!])([A-Z])', r'\1 \2', ai_text)
            ai_text_cleaned = re.sub(r'\s+', ' ', ai_text_cleaned).strip()

            input_text = f"Query: {human_text_cleaned}"
            
            processed_batch['input'].append(input_text)
            processed_batch['output'].append(ai_text_cleaned)

            
        else:
            # Handling Dataset 1 format
            desc = batch['Description'][i]
            patient = batch['Patient'][i]
            doctor = batch['Doctor'][i]
            
            # Clean the text segments
            desc_cleaned = re.sub(r'([.?!])([A-Z])', r'\1 \2', desc)
            desc_cleaned = re.sub(r'\s+', ' ', desc_cleaned).strip()
            
            patient_cleaned = re.sub(r'([.?!])([A-Z])', r'\1 \2', patient)
            patient_cleaned = re.sub(r'\s+', ' ', patient_cleaned).strip()
            
            doctor_cleaned = re.sub(r'([.?!])([A-Z])', r'\1 \2', doctor)
            doctor_cleaned = re.sub(r'\s+', ' ', doctor_cleaned).strip()
            
            # Add to the processed batch
            input_text = f"Query: {desc_cleaned} {patient_cleaned}" if desc_cleaned else f"Query: {patient_cleaned}"
            processed_batch['input'].append(input_text)
            processed_batch['output'].append(doctor_cleaned)
            
    return processed_batch

In [18]:
ds2_train_processed = ds_2["train"].map(process_data_batch, batched=True, num_proc=8, remove_columns=ds_2["train"].column_names)

Map (num_proc=8):   0%|          | 0/106556 [00:00<?, ? examples/s]

In [19]:
ds2_train_processed[0]

{'input': 'Query: Hi. My names Ahdieh. I m from a small city in Iran. My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to life. now he isnot conscious.he is in ICU.his doctor said his conscious rate is 5.what do you think about his condition? If it is needed I can send you his cardiograf and blood test and the drugs they are using for him in ICU. Kindly tell me can we carry him to a better hospital in Tehran(It takes 4 hours to reach there)?',
 'output': 'Hello Addie! Welcome and thank you for asking on Chat Doctor! I passed carefully through your question and would explain that your father clinical situation could be related to prolonged brain hypoxia, due to the cardiac arrest. The time (40 minutes) needed for his resuscitation is long and may have caused irreversible damage to his brain, due to low blood flow (also called hypoxic encephalopathy). For this reason, I recommend consulting with a neurologist and performing a brain MRI and 

In [20]:
ds2_train_processed

Dataset({
    features: ['input', 'output'],
    num_rows: 106556
})

In [21]:
ds2_test_processed = ds_2["test"].map(process_data_batch, batched=True, num_proc=8, remove_columns=ds_2["test"].column_names)

Map (num_proc=8):   0%|          | 0/5609 [00:00<?, ? examples/s]

In [22]:
ds2_test_processed[0]

{'input': 'Query: I wake in the night, usually about 2-3 hours after going to sleep, with both feet and legs to mid calf feeling like they are on fire. slight red discolorization, minor swelling. This is very painful but after getting up, I can walk it off in about 30 minutes.',
 'output': 'Dear patient Here are the possibilities of what you might have.1)PhlebitisPhlebitis means inflammation of the veins, and can cause redness, itching, irritation, pain, and swelling. A simple Doppler can rule this out.2Blood clot in the lifeblood clots in the leg can become very dangerous, symptoms include swelling, redness, tenderness in the leg. Coagulation profile with an angiography may be required3)Cellulitis: Initial stage. Only can be clinically ruled out Hope this helped'}

In [23]:
ds1_processed = health_ds.map(process_data_batch, batched=True, num_proc=8, remove_columns=health_ds.column_names)

Map (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]

In [24]:
ds1_processed[0]

{'input': "Query: Can blood pressure medication be stopped to check improvement in bp levels? I'm 35, BP 150/100 without medicine, never smoke, drink, BMIMy hdl:45, LDL:107, Total Cholesterol:168, Triglyceride:95, Blood Sugar(Fasting):83 (all are without medicine) My question is, should I stop this medicine to see the afffect. it is worth to mention that recently I've increased my physical activity,",
 'output': 'Hello,Thanks for writing to Health Care Magic, I am Dr Asad Riaz, I have closely read your question and I understand your concerns, I will hereby guide you regarding your health related problem. BP is one major risk factor for major complication like MI or stroke...if pt have high BP as u r having ..1st we need to do life style modification to see whteher it work to de BP or not in whch i advice pt to lower salt intake,drinkng dec weight n exercise for a period of 6 month n still if pt have high BP then need to add med to control it so we prevent major comlication..as u having

In [25]:
ds1_processed

Dataset({
    features: ['input', 'output'],
    num_rows: 3000
})

In [26]:
split_ds = ds1_processed.train_test_split(test_size=0.05, seed=42)
ds1_train = split_ds["train"]
ds1_eval  = split_ds["test"]

split_ds

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 2850
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 150
    })
})

In [27]:
# Now, combine and process your datasets correctly and efficiently
from datasets import concatenate_datasets

seed=42

# Concatenate the processed datasets
combined_train = concatenate_datasets([ds1_train, ds2_train_processed]).shuffle(seed=seed)
combined_eval = concatenate_datasets([ds1_eval, ds2_test_processed]).shuffle(seed=seed)

combined_train

Dataset({
    features: ['input', 'output'],
    num_rows: 109406
})

In [28]:
combined_eval

Dataset({
    features: ['input', 'output'],
    num_rows: 5759
})

In [29]:
SYSTEM_PROMPT = (
    "You are an empathetic medical assistant. Provide general health information, "
    "not a diagnosis. Encourage consulting a clinician for urgent or uncertain cases."
)

def format_chat(data, tokenizer):

    combined_user_content = f"{SYSTEM_PROMPT}\n\n{data['input']}"

    row_data = [

        {"role": "user", "content": combined_user_content},
        {"role": "model", "content": data["output"]},
    ]
    return tokenizer.apply_chat_template(row_data, tokenize=False)

In [33]:
training_args = SFTConfig(
    output_dir="/content/gemma-2b-it-health-convo-big",
    # reduce gradient noise (keep per_device=1 if memory-bound)
    per_device_train_batch_size=2,        # keep trying different batch size to ensure you're maximizing your GPU memory capability
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,        
    gradient_checkpointing=True,          

    learning_rate=1e-6,                  
    lr_scheduler_type="cosine_with_restarts",            
    lr_scheduler_kwargs={"num_cycles": 2},
    warmup_ratio=0.01,                     # add warmup to smooth early steps
    num_train_epochs = 3,

    # regularization & stability
    weight_decay=0.05,                     # improve generalization a bit
    max_grad_norm=1.0,                     # keep default clipping explicit
    optim="adamw_torch_fused",             # faster / stabler on recent CUDA Always use fused because it get rids of overhead in the kernel by fusing them
    adam_beta2=0.95,                       # quicker adaptation during SFT

    bf16=True,                             
    logging_strategy="steps",
    logging_steps=10,                     
    eval_strategy="steps",
    eval_steps=500,                        
    save_strategy="steps",
    save_steps=500,                        
    save_total_limit=2,

    # sequence packing → more tokens/step, better convergence per wall-clock
    dataset_kwargs={"packing": True},
    # group_by_length=True,
    max_length=512,

    push_to_hub=True,
    hub_model_id=NEW_MODEL_REPO,
    hub_strategy="every_save",
    load_best_model_at_end=True,
)

In [34]:
trainer = SFTTrainer(
    model=model,
    train_dataset=combined_train,
    eval_dataset=combined_eval,
    processing_class=tokenizer,   
    # data_collator=data_collator,  #huggingface
    formatting_func=lambda x: format_chat(x, tokenizer),
    args=training_args,
)

Applying formatting function to train dataset:   0%|          | 0/109406 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/109406 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/109406 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/109406 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/5759 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/5759 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/5759 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5759 [00:00<?, ? examples/s]

In [35]:
!rm -rf /content/gemma-2b-it-health-convo-big

In [36]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [37]:
import wandb

wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")

wandb.login(key=wandb_key)

wandb.init(project=WANDB_PROJECT)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: emmy_wan (emmy_wan-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [38]:
# from transformers import EarlyStoppingCallback
# trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=1e-4))

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
500,3.129900,3.143991
1000,2.775700,2.786569
1500,2.583100,2.648035
2000,2.624800,2.583095
2500,2.470300,2.541324
3000,2.488800,2.508859
3500,2.470900,2.482572
4000,2.469300,2.462054
4500,2.306900,2.445911


In [ ]:
trainer.train(resume_from_checkpoint=True)

In [ ]:
from peft import LoraConfig, PeftModel, get_peft_model

In [ ]:
model.push_to_hub("Yanmife/gemma-2b-it-health-convo")

In [ ]:
trainer.push_to_hub("Yanmife/gemma-2b-it-health-convo")

In [ ]:
# load model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

model_name = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained(NEW_MODEL_REPO)
model = PeftModel.from_pretrained(model, NEW_MODEL_REPO)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
model.eval()

In [ ]:
# Create a dictionary for the new input
new_input = {
    "Description": "I'm a 35-year-old male with a history of hypertension. I've been having headaches and dizziness for the past week. My blood pressure readings have been higher than usual.",
    "Patient": "What could be causing this? I'm worried it's something serious.",
}

# Format the input using the same structure from your training function
row_json = [
    {"role": "user", "content": f"Description: {new_input['Description']}\nPatient: {new_input['Patient']}"},
]

# Apply the chat template to create the final prompt string
formatted_prompt = tokenizer.apply_chat_template(
    row_json,
    tokenize=False,
    add_generation_prompt=True, # This adds the special tokens for the model to start generating
)

In [ ]:
# Tokenize the formatted prompt
input_ids = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

# Generate the response
outputs = model.generate(
    **input_ids,
    max_new_tokens=512, # You can adjust this for longer or shorter responses
    do_sample=True,      # This enables sampling for more varied responses
    temperature=0.7,     # Controls the "creativity"
    top_k=50,            # Controls the token choices
    top_p=0.95,          # Another method to control token choices
    pad_token_id=tokenizer.eos_token_id # Prevents generation from stopping prematurely
)

# Decode the generated tokens back into a string
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the full conversation to see the model's response
print(response)